In [1]:
# Import libraries
import itertools
import string
import shutil
import time
start_time = time.time()

In [2]:
# Input and output file paths
LATIN_FILES_BASEPATH = "Input/3_gram_input/"
LEMMATIZER_PATH = "new_lemmatizer.csv"
NO_OF_FILES = 1
OUTPUT_FILE = "Output/3_gram_output_"+str(NO_OF_FILES)

In [3]:
basepath = LATIN_FILES_BASEPATH
fileList = []
for f in os.listdir(basepath):
    fileList.append(f)
fileList = sorted(fileList)

textFile = sc.textFile(basepath + "/" + fileList[0])
for f1 in fileList[1:NO_OF_FILES]:
    textFile = textFile.union(sc.textFile(basepath + "/" + f1))


In [4]:
lemmaDict = {}

In [5]:
def readLemmas():
    with open(LEMMATIZER_PATH) as f1:
        for line in f1.readlines():
            lemmas = line.split(',')
            lemmaDict[lemmas[0]] = list()
            for word in lemmas[1:]:
                if word=='':
                    break
                lemmaDict[lemmas[0]].append(word)

In [6]:
readLemmas()

In [7]:
def checkWord(line):
    if (len(line)>0):
        position = line.split("> ")[0]+">"
        words=line.split("> ")[1].lower().translate(str.maketrans('','',string.punctuation)).split()
        fullList =[]
        for i in itertools.combinations(words,3):
            a = list(i)
            word1=i[0]
            word2=i[1]
            word3=i[2]
            l1 =l2=l3=[]
            if word1 in lemmaDict:
                l1 = lemmaDict.get(word1)
            else:
                l1=[word1]
            if word2 in lemmaDict:
                l2 = lemmaDict.get(word2)
            else:
                l2=[word2]
            if word3 in lemmaDict:
                l3 = lemmaDict.get(word3)
            else:
                l3=[word3]
            for lemma_comb in list(itertools.product(l1,l2,l3)):
                fullList.append((position,tuple(sorted(lemma_comb)),))
        return fullList

In [8]:
lines = textFile.map(checkWord)

In [9]:
#lines.collect()

In [10]:
keyValues = lines.flatMap(lambda line:line)

In [11]:
#keyValues.collect()

In [12]:
def processTuple(tuple1):
    return tuple1[1],[tuple1[0]]

In [13]:
mapper = keyValues.map(processTuple)

In [14]:
#mapper.collect()

In [15]:
reducer = mapper.reduceByKey(lambda a,b: a+b)

In [16]:
#reducer.collect()

In [17]:
if os.path.exists(OUTPUT_FILE):
    shutil.rmtree(OUTPUT_FILE)
reducer.saveAsTextFile(OUTPUT_FILE)

In [18]:
print("--- %s seconds ---" % round(time.time() - start_time, 2))

--- 22.99 seconds ---
